# 3DRDNN

Main pipeline for research in my Master.

## goals in this notebook

Prepare the pipeline for any 3d DNN to train on CT data.
1) data loader
2) DNN
3) Training
4) Results comparison

In [1]:
# Testing reading GPU
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
import os
from matplotlib import pyplot as plt
import numpy as np
import glob
# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print(tf.test.gpu_device_name())

TensorFlow Version: 2.8.0
/device:GPU:0


In [2]:
# Data loader
from utils import data_loader
import random



loader = data_loader.NiiDataLoader()
"""
#random.shuffle(files_segmenation)
#random.shuffle(files_volume)

train_volume = files_volume[0:22]
train_segmentation = files_segmenation[0:22]

print(train_volume)
print(train_segmentation)

valid_volume = files_volume[22:]
valid_segmentation = files_segmenation[22:]

print(valid_volume)
print(valid_segmentation)


"""

train_generator = loader.data_generator_2d(0)
valid_generator = loader.data_generator_2d(1)
counter_train = 11391
counter_valid = 3527

print(type(train_generator))
"""
for x in train_generator:
    counter_train+=1

for x in valid_generator:
    counter_valid+=1"""

"""<class 'generator'> 
counter_train 3372
counter_valid 1138"""
counter_train = 0
counter_valid = 0
for x in train_generator:
    counter_train+=1
for x in valid_generator:
    counter_valid+=1
print(counter_train)
print(counter_valid)

<class 'generator'>
3372
1138


In [3]:
#datasets for tf
batch_size = 8
dataset = tf.data.Dataset.from_generator(
     loader.data_generator_2d,
     (tf.float32,tf.float32), 
    (tf.TensorShape([256,256,1]), tf.TensorShape([256,256,2])), args = ([0]))
dataset = dataset.repeat(101).batch(batch_size)

valid_dataset = tf.data.Dataset.from_generator(
     loader.data_generator_2d,
     (tf.float32,tf.float32), 
    (tf.TensorShape([256,256,1]), tf.TensorShape([256,256,2])),args = ([1]))
valid_dataset = valid_dataset.repeat(100).batch(batch_size) #.shuffle(1000,  reshuffle_each_iteration=True)


print(dataset)
print(valid_dataset)

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 2), dtype=tf.float32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 2), dtype=tf.float32, name=None))>


In [4]:
# Network set-up
from utils import models

model = models.model_call(model_name="2DUNET",px=256,features=64)

In [5]:
# Metrics and training

import tensorflow.keras.backend as K
def dice(y_true, y_pred):
    smooth = 1
    intersection = K.sum(y_true * y_pred, axis=list(range(1, K.ndim(y_true))))
    union = K.sum(y_true, axis=list(range(1, K.ndim(y_true)))) + K.sum(y_pred, axis=list(range(1, K.ndim(y_true))))
    return K.mean((2. * intersection + smooth) / (union + smooth), axis=0)

adam = tf.keras.optimizers.Adam(
    learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')
precision = tf.keras.metrics.Precision(
    thresholds=0.5, class_id=1)
recall = tf.keras.metrics.Recall(
    thresholds=0.5, class_id=1)

model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=[precision,recall])
model.summary(positions=[.33, .66, .78, 1.])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape                    Param #     Connected to          
 input_1 (InputLayer)           [(None, 256, 256, 1)]           0           []                    
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64)            640         ['input_1[0][0]']     
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64)            36928       ['conv2d[0][0]']      
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 128, 128, 64)            0           ['conv2d_1[0][0]']    
                                                                                              

In [6]:
# Results
# reading checkpoint if needen
#
#################################

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,TerminateOnNaN
# training
callbacks = [
    #EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.0000001, verbose=1),
    ModelCheckpoint("2DUNET_liver_test.h5", verbose=1, save_best_only=True, save_weights_only=True),
    TerminateOnNaN()]
epochs = 10
steps_per_epoch = counter_train / batch_size #ilość kawałków x plików / batch_size* z dokładnościa do dziielnia przez 32

validations_steps = counter_valid / batch_size #ilość kawałków x plików /batch_size
history = model.fit(dataset,validation_data=valid_dataset, callbacks=callbacks, initial_epoch=0,
                    epochs=epochs, validation_steps = validations_steps, steps_per_epoch=steps_per_epoch
                    )

Epoch 1/10
 91/421 [=====>........................] - ETA: 2:10 - loss: 7127.6753 - precision: 0.0734 - recall: 0.0354